In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `JANI-CHATBOT` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `JANI-

In [ ]:
!pip install fastapi uvicorn pyngrok
# !pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00


In [ ]:
!ngrok config add-authtoken "" #create and put your ngrok api key here

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Try with these for the open chat --
1.What skills do I need to become a Machine Learning Engineer?
Provide key statistics about the job market for Data Scientists in 2024.
2.what is ML
3.Give me a quick summary of Linear Regression.
4.Compare job opportunities for Data Analysts vs. Software Engineers.
5.

In [ ]:


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import re

nest_asyncio.apply()
app = FastAPI()

# Enable CORS
origins = ["http://localhost:5173","http://localhost:5174"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ✅ Load Fine-Tuned Model Correctly Shreyashs12345/JANI_AI_CHATBOT_2
fine_tuned_model_name = "Shreyashs12345/JANI_AI_CHATBOT_2"
finetuned_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_name)
finetuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
finetuned_model.to(device)

# ✅ Load Base Model (General Chat)
base_model_name = "microsoft/phi-1_5"
general_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
general_model = AutoModelForCausalLM.from_pretrained(base_model_name).to(device)

class QueryRequest(BaseModel):
    query: str

# ✅ Fine-Tuned Model Chat Endpoint
@app.post("/ask")
async def chat_with_finetuned_ai(request: QueryRequest):
    try:
        input_text = request.query
        inputs = finetuned_tokenizer(input_text, return_tensors="pt").to(device)

        output = finetuned_model.generate(
            **inputs,
            max_length=100,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=finetuned_tokenizer.eos_token_id,
            repetition_penalty=1.2
        )

        response = finetuned_tokenizer.decode(output[0], skip_special_tokens=True).strip()
        response = response.replace(input_text, "").strip()
        response = response.replace("Output:", "").strip()

        match = re.search(r"Output:\s*(.+)", response)
        if match:
            response = match.group(1).strip()

        return {"response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# ✅ General Model Chat Endpoint
@app.post("/general_chat")
async def chat_with_general_ai(request: QueryRequest):
    try:
        input_text = f"User: {request.query}\nAssistant:"
        inputs = general_tokenizer(input_text, return_tensors="pt").to(device)

        output = general_model.generate(
            **inputs,
            max_length=200,
            temperature=0.7,
            top_p=0.9,
            eos_token_id=general_tokenizer.eos_token_id,
            repetition_penalty=1.2
        )

        response = general_tokenizer.decode(output[0], skip_special_tokens=True).strip()
        response = response.split("Assistant:")[-1].strip()

        return {"response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# ✅ Expose API via ngrok
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")

# ✅ Run Uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     2405:201:d058:d839:38e3:f084:654c:ec0d:0 - "POST /general_chat HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     2405:201:d058:d839:38e3:f084:654c:ec0d:0 - "POST /general_chat HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [250]
